In [1]:
from mts.core.mtserie_dataset import MTSerieDataset
from mts.core.projections import ProjectionAlg
from models.emotion_dataset_controller import *
import matplotlib.pyplot as plt
from mts.core.projections import ProjectionAlg, euclidean_distance_matrix, mds_projection, compute_k_distance_matrixes, compute_distance_matrix
import tensorflow as tf


config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.gpu_options.allow_growth = True
tf.keras.backend.set_session(tf.Session(config=config));


import sys
sys.path.insert(1, '/home/texs/Documents/AirQuality/repositories/peax/experiments')


In [2]:
controller = AppController()

In [3]:
# dataset_id = 'emotions_in_music'
dataset_id = 'afew_va'
controller.loadLocalDataset(dataset_id)

True

In [4]:

dataset = controller.datasets[dataset_id]
X = dataset.values()

print(X.shape)
print(X.max())
print(X.min())

(418, 34, 2)
0.95
0.05


In [6]:
from sklearn.preprocessing import MinMaxScaler

def normalizeWindow(data, percentile: float = 99.9):
    cutoff = np.percentile(data, (0, percentile))
    data_norm = np.copy(data)
    data_norm[np.where(data_norm < cutoff[0])] = cutoff[0]
    data_norm[np.where(data_norm > cutoff[1])] = cutoff[1]

    return MinMaxScaler().fit_transform(data_norm)

X = dataset.values()
# X = np.array([  normalizeWindow(np.concatenate([x, np.zeros([1, 2])]) )    for x in X])
# X = np.array([(np.concatenate([x, np.zeros([1, 2])]) )    for x in X])


N, T, D =X.shape
# allWindows = np.split(X, 2, axis = 2)
# windows1 = allWindows[0]
# print(windows1.shape)
# print(windows2.shape)

In [7]:
windows = X
n = windows.shape[0]
train_index = int(n* 0.6)
test_index = train_index + int(n* 0.2)
train = windows[0: train_index]
test = windows[train_index: test_index]
val = windows[test_index: ]
print(train.shape)
print(test.shape)
print(val.shape)

DATA_FILEPATH = f'savedData/data.npy'
with open(DATA_FILEPATH, 'wb') as f:
    np.save(f, train)
    np.save(f, test)
    np.save(f, val)


(250, 34, 2)
(83, 34, 2)
(85, 34, 2)


In [8]:
settings = {
  "conv_filters": [
    128,
    192,
    288,
    432
  ],
  "conv_kernels": [
    3,
    5,
    7,
    9
  ],
  "dense_units": [
    1024,
    256
  ],
  "dropouts": [
    0,
    0,
    0,
    0,
    0,
    0
  ],
  "embedding": 25,
  "reg_lambda": 0,
  "optimizer": "adadelta",
  "learning_rate": 1.0,
  "learning_rate_decay": 0.001,
  "loss": "bce",
  "metrics": [],
  "batch_norm": [
    False,
    False,
    False,
    False,
    False,
    False
  ],
  "batch_norm_input": False,
}

In [9]:
DATA_FILEPATH = f'savedData/data.npy'
with open(DATA_FILEPATH, 'rb') as f:
    train_data = np.load(f, allow_pickle=True)
    test_data = np.load(f, allow_pickle=True)
    val_data = np.load(f, allow_pickle=True)


In [17]:
from nn_model import create_model

def trainEncoder(data_train: np.array, 
         data_test: np.array, 
         data_val: np.array,
         model_name:str,
         window_size: int,
         batch_size: int,
         n_epochs: int,
         settings: dict,
):
#     _, _, autoencoder = create_model(window_size, **settings)
    _, _, autoencoder = create_model(input_dim=T, n_features=D, summary=True)
    history = autoencoder.fit(
        data_train,
        data_train,
        epochs=n_epochs,
        batch_size=batch_size,
        shuffle=True,
        validation_data=(data_val, data_val),
        # sample_weight=sample_weight_train,
        verbose=True,
        # callbacks=callbacks,
    )
    autoencoder.save(model_name)
    return history



In [18]:
# model.fit(train_data,train_data)
print(train_data.shape)

(250, 34, 2)


In [19]:
# from air_quality import trainEncoder
# import importlib
# importlib.reload(air_quality)

history = trainEncoder(
    train_data,
    test_data, 
    val_data, 
    model_name = 'savedData/test.h5', 
    window_size = T,
    batch_size = 100,
    n_epochs = 120,
    settings = settings,
    
)

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 34, 2)             0         
_________________________________________________________________
conv0 (Conv1D)               (None, 17, 120)           2280      
_________________________________________________________________
flatten (Flatten)            (None, 2040)              0         
_________________________________________________________________
fc1 (Dense)                  (None, 256)               522496    
_________________________________________________________________
fc2 (Dense)                  (None, 64)                16448     
_________________________________________________________________
fc3 (Dense)                  (None, 16)                1040      
_________________________________________________________________
embed (Dense)                (None, 10)                170 

ValueError: Error when checking target: expected output to have shape (240, 2) but got array with shape (34, 2)